### Load Data

In [1]:
import pandas as pd

path = 'data/train.csv'
df = pd.read_csv(path)
df.head()

,id,model_a,model_b,prompt,response_a,response_b,winner_model_a,winner_model_b,winner_tie
0,30192,gpt-4-1106-preview,gpt-4-0613,"[""Is it morally right to try to have a certain...","[""The question of whether it is morally right ...","[""As an AI, I don't have personal beliefs or o...",1,0,0
1,53567,koala-13b,gpt-4-0613,"[""What is the difference between marriage lice...","[""A marriage license is a legal document that ...","[""A marriage license and a marriage certificat...",0,1,0
2,65089,gpt-3.5-turbo-0613,mistral-medium,"[""explain function calling. how would you call...","[""Function calling is the process of invoking ...","[""Function calling is the process of invoking ...",0,0,1
3,96401,llama-2-13b-chat,mistral-7b-instruct,"[""How can I create a test set for a very rare ...","[""Creating a test set for a very rare category...","[""When building a classifier for a very rare c...",1,0,0
4,198779,koala-13b,gpt-3.5-turbo-0314,"[""What is the best way to travel from Tel-Aviv...","[""The best way to travel from Tel Aviv to Jeru...","[""The best way to travel from Tel-Aviv to Jeru...",0,1,0


### Download/Load Weights

In [2]:
import torch
from transformers import DistilBertTokenizer, DistilBertModel, DistilBertConfig
import os

weights = 'pretrained'
model_dir = "weights/distilbert"

if not os.path.isdir(model_dir):
    os.makedirs(model_dir)

device = torch.device("cuda" if torch.cuda.is_available() else
                      "xpu" if torch.xpu.is_available() else "cpu")
print(f"Using device: {device}")

# Option A: Download and save pretrained weights
def download_and_save():
    model = DistilBertModel.from_pretrained("distilbert-base-uncased")
    model.save_pretrained(model_dir)
    print("Downloaded and saved pretrained DistilBERT weights.")
    return model

# Option B: Load saved weights (if available)
def load_saved():
    model = DistilBertModel.from_pretrained(model_dir)
    print("Loaded saved DistilBERT weights.")
    return model

# Option C: Randomly initialize weights
def init_random():
    config = DistilBertConfig()
    model = DistilBertModel(config)
    print("Initialized DistilBERT with random weights.")
    return model

# Load weights
if os.path.exists(os.path.join(model_dir, "model.safetensors")):
    base_model = load_saved()
else:
    base_model = download_and_save() if weights == 'pretrained' else init_random()

# --- Model and Tokenizer ---
tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")
print(f"Model type: {type(base_model).__name__}")
print(f"Tokenizer vocab size: {tokenizer.vocab_size}")
print(f"Hidden size: {base_model.config.dim}, Layers: {base_model.config.n_layers}, Heads: {base_model.config.n_heads}")

# --- Inference QC ---
sample_text = "I love using Hugging Face Transformers!"
inputs = tokenizer(sample_text, return_tensors="pt").to(device)
base_model = base_model.to(device)

with torch.no_grad():
    outputs = base_model(**inputs)

print(f"Inference QC: last_hidden_state shape {outputs.last_hidden_state.shape}")

# --- Training QC ---
base_model.train()
inputs = tokenizer(list(df["prompt"][:2]), return_tensors="pt", padding=True, truncation=True).to(device)
optimizer = torch.optim.AdamW(base_model.parameters(), lr=1e-5)

optimizer.zero_grad()
outputs = base_model(**inputs)
loss = outputs.last_hidden_state.mean()  # dummy scalar loss just for gradient test
loss.backward()
optimizer.step()
print(f"Training QC: backward pass successful (dummy loss={loss.item():.4f})")


c:\Users\zacco\OneDrive\Documents\Personal\Code\venvs\xpu\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Using device: xpu
Loaded saved DistilBERT weights.
Model type: DistilBertModel
Tokenizer vocab size: 30522
Hidden size: 768, Layers: 6, Heads: 12
Inference QC: last_hidden_state shape torch.Size([1, 9, 768])
Training QC: backward pass successful (dummy loss=-0.0097)


### Dataset/Dataloader

In [78]:
import torch
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split

class ResponseDataset(Dataset):
    def __init__(self, df, max_length=512):
        self.df = df
        self.max_length = max_length

    def outcome_to_class(self, row):
        if row["winner_model_a"] == 1:
            return 2   # A wins
        if row["winner_model_b"] == 1:
            return 0   # B wins
        return 1       # tie

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]

        tokens_a = tokenizer(
            row["response_a"], max_length=self.max_length,
            padding="max_length", truncation=True, return_tensors="pt"
        )
        tokens_b = tokenizer(
            row["response_b"], max_length=self.max_length,
            padding="max_length", truncation=True, return_tensors="pt"
        )

        label = torch.tensor(self.outcome_to_class(row), dtype=torch.long)

        return {
            "input_ids_a": tokens_a["input_ids"].squeeze(0),
            "attention_mask_a": tokens_a["attention_mask"].squeeze(0),
            "input_ids_b": tokens_b["input_ids"].squeeze(0),
            "attention_mask_b": tokens_b["attention_mask"].squeeze(0),
            "label": label,
        }

df_train, df_temp = train_test_split(df, test_size=0.1, random_state=42, shuffle=True)
df_val, df_test = train_test_split(df_temp, test_size=0.5, random_state=42, shuffle=True)

# Create datasets
train_dataset = ResponseDataset(df_train)
val_dataset = ResponseDataset(df_val)
test_dataset = ResponseDataset(df_test)

# Create dataloaders
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

### Model

In [31]:
import torch.nn as nn

class ResponseScorer(nn.Module):
    def __init__(self, base_model, freeze_base=False, device="cpu"):
        super().__init__()
        self.base_model = base_model
        hidden_dim = self.base_model.config.dim

        # classifier head → outputs logits for [B wins, Tie, A wins]
        self.head = nn.Sequential(
            nn.Linear(hidden_dim * 2, hidden_dim),
            nn.ReLU(),
            nn.Dropout(0.1),
            nn.Linear(hidden_dim, hidden_dim // 2),
            nn.ReLU(),
            nn.Dropout(0.1),
            nn.Linear(hidden_dim // 2, 3)   # 3-class output
        )

        if freeze_base:
            for p in self.base_model.parameters():
                p.requires_grad = False

        self.to(device)
        self.device = device

    def encode(self, input_ids, attention_mask):
        out = self.base_model(input_ids=input_ids, attention_mask=attention_mask)
        return out.last_hidden_state[:, 0, :]  # CLS

    def forward(self, input_ids_a, mask_a, input_ids_b, mask_b):
        # embeddings from base model
        h_a = self.encode(input_ids_a.to(self.device), mask_a.to(self.device))
        h_b = self.encode(input_ids_b.to(self.device), mask_b.to(self.device))

        # concatenate the two embeddings
        h = torch.cat([h_a, h_b], dim=-1)

        # 3-class logits
        logits = self.head(h)
        return logits

### Train

In [32]:
import torch
import os

mode = 'test' # 'train'

model = ResponseScorer(base_model, freeze_base=False, device=device)
loss_fn = nn.CrossEntropyLoss()

if mode == 'train':

    num_epochs = 3
    val_losses, val_accuracies = [], []

    # Create directory for checkpoints
    if not os.path.isdir("checkpoints"):
        os.makedirs("checkpoints", exist_ok=True)
    save_every = 100  # batches

    for epoch in range(1, num_epochs + 1):
        model.train()
        running_loss = 0.0
        total_batches = len(train_loader)

        for i, batch in enumerate(train_loader, 1):
            input_ids_a = batch["input_ids_a"].to(device)
            mask_a = batch["attention_mask_a"].to(device)
            input_ids_b = batch["input_ids_b"].to(device)
            mask_b = batch["attention_mask_b"].to(device)
            labels = batch["label"].to(device)        # <-- single class ID 0/1/2

            optimizer.zero_grad()

            # logits: [batch, 3]
            logits = model(input_ids_a, mask_a, input_ids_b, mask_b)

            loss = loss_fn(logits, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

            # ---- progress indicator ----
            pct_done = 100 * i / total_batches
            print(
                f"\rEpoch {epoch}: {pct_done:.1f}% done | Avg Loss: {running_loss / i:.4f}",
                end=""
            )

            # ---- periodic save ----
            if i % save_every == 0:
                ckpt_path = f"checkpoints/epoch{epoch}_batch{i}.pt"
                torch.save(model.state_dict(), ckpt_path)
                print(f"\nSaved checkpoint to {ckpt_path}")

        print()  # newline after epoch progress

        # Save final epoch weights
        torch.save(model.state_dict(), f"checkpoints/epoch{epoch}_final.pt")

        # ----------------------
        # Validation
        # ----------------------
        model.eval()
        val_loss = 0.0
        correct = 0
        total = 0

        with torch.no_grad():
            for batch in val_loader:
                input_ids_a = batch["input_ids_a"].to(device)
                mask_a = batch["attention_mask_a"].to(device)
                input_ids_b = batch["input_ids_b"].to(device)
                mask_b = batch["attention_mask_b"].to(device)
                labels = batch["label"].to(device)

                logits = model(input_ids_a, mask_a, input_ids_b, mask_b)

                val_loss += loss_fn(logits, labels).item()

                preds = logits.argmax(dim=-1)
                correct += (preds == labels).sum().item()
                total += labels.size(0)

        avg_val_loss = val_loss / len(val_loader)
        val_acc = correct / total

        val_losses.append(avg_val_loss)
        val_accuracies.append(val_acc)

        print(f"Epoch {epoch} done — Val Loss: {avg_val_loss:.4f}, Val Acc: {val_acc:.4f}")

### Test

In [79]:
path_pretrain = 'checkpoints/epoch3_batch900.pt'

# Instantiate model + scorer
if path_pretrain is not None:
    model = ResponseScorer(base_model, freeze_base=False, device=device)
    model.load_state_dict(torch.load(path_pretrain, map_location=device))
    model.to(device)
else:
    model = ResponseScorer(base_model, freeze_base=False, device=device)

model.eval()
correct = 0
total = 0

num_batches = len(test_loader)

with torch.no_grad():
    for i, batch in enumerate(test_loader, 1):
        input_ids_a = batch["input_ids_a"].to(device)
        mask_a = batch["attention_mask_a"].to(device)
        input_ids_b = batch["input_ids_b"].to(device)
        mask_b = batch["attention_mask_b"].to(device)

        labels = batch["label"].to(device)
        preds = logits.argmax(dim=-1)

        correct += (preds == labels).sum().item()
        total += labels.size(0)

        pct = 100 * i / num_batches
        print(f"[{pct:6.2f}%] Running Test Accuracy: {correct / total:.4f}")


[  1.11%] Running Test Accuracy: 0.5000
[  2.22%] Running Test Accuracy: 0.3906
[  3.33%] Running Test Accuracy: 0.3333
[  4.44%] Running Test Accuracy: 0.3125
[  5.56%] Running Test Accuracy: 0.3125
[  6.67%] Running Test Accuracy: 0.3073
[  7.78%] Running Test Accuracy: 0.3393
[  8.89%] Running Test Accuracy: 0.3242
[ 10.00%] Running Test Accuracy: 0.3368
[ 11.11%] Running Test Accuracy: 0.3312
[ 12.22%] Running Test Accuracy: 0.3267
[ 13.33%] Running Test Accuracy: 0.3229
[ 14.44%] Running Test Accuracy: 0.3245
[ 15.56%] Running Test Accuracy: 0.3281
[ 16.67%] Running Test Accuracy: 0.3271
[ 17.78%] Running Test Accuracy: 0.3242
[ 18.89%] Running Test Accuracy: 0.3272
[ 20.00%] Running Test Accuracy: 0.3316
[ 21.11%] Running Test Accuracy: 0.3355
[ 22.22%] Running Test Accuracy: 0.3422
[ 23.33%] Running Test Accuracy: 0.3378
[ 24.44%] Running Test Accuracy: 0.3338
[ 25.56%] Running Test Accuracy: 0.3288
[ 26.67%] Running Test Accuracy: 0.3320
[ 27.78%] Running Test Accuracy: 0.3362


### Kaggle Evaluation

In [67]:
df_test = pd.read_csv('data/test.csv')
df_test.head()

,id,prompt,response_a,response_b
0,136060,"[""I have three oranges today, I ate an orange ...","[""You have two oranges today.""]","[""You still have three oranges. Eating an oran..."
1,211333,"[""You are a mediator in a heated political deb...","[""Thank you for sharing the details of the sit...","[""Mr Reddy and Ms Blue both have valid points ..."
2,1233961,"[""How to initialize the classification head wh...","[""When you want to initialize the classificati...","[""To initialize the classification head when p..."


In [66]:
def kaggle_to_datset(row, max_length = 512):

    tokens_a = tokenizer(
                row["response_a"], max_length=max_length,
                    padding="max_length", truncation=True, return_tensors="pt"
                )
    tokens_b = tokenizer(
        row["response_b"], max_length=max_length,
        padding="max_length", truncation=True, return_tensors="pt"
    )

    return [tokens_a["input_ids"].reshape(1,-1), 
            tokens_a["attention_mask"].reshape(1,-1), 
            tokens_b["input_ids"].reshape(1,-1), 
            tokens_b["attention_mask"].reshape(1,-1),]

with torch.no_grad():
    for i in range(df_test.shape[0]):
        test_set = kaggle_to_datset(df_test.iloc[i])
        logits = model(*test_set)
        preds = logits.argmax(dim=-1)
        print(f'prediction is {preds} for test set {i}')

prediction is tensor([1], device='xpu:0') for test set 0
prediction is tensor([1], device='xpu:0') for test set 1
prediction is tensor([0], device='xpu:0') for test set 2
